In [3]:
#필요한 모듈 임포트하기
import pandas as pd
from pandas import DataFrame  as df
import numpy as np
from collections import Counter
from konlpy.tag import Okt
okt = Okt()
%matplotlib inline
import matplotlib.pyplot as plt
import re 
import gensim
from gensim import corpora, models
from gensim.models import CoherenceModel 

In [4]:
data = pd.read_csv('data\저출산 정책 기사_댓글_모음.csv', encoding='cp949')
data.head()

,reply,tokenized,policy
0,무슨 중복이요 그럼 휴직급여 그거 하나 받고 키우라는건디 다 일 그만두라는거에요 ...,"['중복', '요', '휴직급', '그거', '키우', '건디', '그만두', '에...",영아수당 지급
1,와 년 주는거도 건드리네 더줄생각은 못하고,"['건드리', '생각']",영아수당 지급
2,무슨 중복인지 그럼 다 경력단절되어야하는건가,"['중복', '인지', '경력', '단절', '어야', '건가']",영아수당 지급
3,세금 내고 일하는 워킹맘이 죄인인가 부모급여 줄 것처럼 언플해서 원래 지자체에서 자...,"['세금', '워킹', '맘', '죄인', '인가', '부모', '급여', '처럼'...",영아수당 지급
4,기사놈 사회성 없는거같음,"['기사', '사회', '성']",영아수당 지급


In [5]:
text = data[data['policy'] == '영아수당 지급'].iloc[:,[0,2]]
text

,reply,policy
0,무슨 중복이요 그럼 휴직급여 그거 하나 받고 키우라는건디 다 일 그만두라는거에요 ...,영아수당 지급
1,와 년 주는거도 건드리네 더줄생각은 못하고,영아수당 지급
2,무슨 중복인지 그럼 다 경력단절되어야하는건가,영아수당 지급
3,세금 내고 일하는 워킹맘이 죄인인가 부모급여 줄 것처럼 언플해서 원래 지자체에서 자...,영아수당 지급
4,기사놈 사회성 없는거같음,영아수당 지급
...,...,...
2073,뭔 돈으로 준당가,영아수당 지급
2074,육아휴직 안해주는 회사는 어떡합니까그지같은 굥아,영아수당 지급
2075,애시기 낳으면 애국이라고 지껄이는것들 ㅇ나게 많네 명이상 낳는 부모들한테나 지급해라,영아수당 지급
2076,문재인정부 년간의 나랏빚 도덕적해이 기초연금폐지하고 이런자들만 독식해온 근로장려금폐...,영아수당 지급


### 데이터 정리

In [6]:
# 데이터 프레임의 'text' 열의 값들을 str 형식으로 바꾸기
text.reply = text.reply.astype(str)

#데이터 프레임의 'text' 열의 값 중 keyword1이나 keyword 2가 포함된 행만 Data에 저장
clean_Data = text.loc[text['reply'].str.contains('keyword1|keyword2')]

#데이터 프레임의 'text' 열의 값 중 keyword1이나 keyword 2가 포함된 행은 삭제
clean_Data = text[~text['reply'].str.contains('keyword1|keyword2')]

#text와 timestamp 열을 기준으로 중복된 데이터를 삭제, inplace : 데이터 프레임을 변경할지 선택(원본을 대체)
clean_Data.drop_duplicates(subset=['reply','policy'], inplace=True)

#한글이 아니면 빈 문자열로 바꾸기
clean_Data['reply'] = clean_Data['reply'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣]',' ',regex=True)

#빈 문자열 NAN 값으로 바꾸기
clean_Data = clean_Data.replace({'': np.nan})
clean_Data = clean_Data.replace(r'^\s*$', None, regex=True)

#NAN 이 있는 행은 삭제
clean_Data.dropna(how='any', inplace=True)

#인덱스 차곡차곡
clean_Data = clean_Data.reset_index (drop = True)

#데이터 프레임에 null 값이 있는지 확인
print(clean_Data.isnull().values.any()) 

False


### 2. 데이터 전 처리하기

In [7]:
#텍스트 데이터를 리스트로 변환
Data_list=clean_Data.reply.values.tolist()

#리스트를 요소별로(트윗 하나) 가져와서 명사만 추출한 후 리스트로 저장
data_word=[]
for i in range(len(Data_list)):
    try:
        data_word.append(Okt().nouns(Data_list[i])) ## Okt.nouns() 오류 발생 --> Okt().nouns() 으로 변경
    except Exception as e:
        continue

#트윗에서 명사만 추출해서 만든 리스트
data_word

[['무슨',
  '중복',
  '휴직',
  '급여',
  '그거',
  '하나',
  '건디',
  '일',
  '거',
  '원금',
  '평생',
  '유아기',
  '잠시',
  '도',
  '이',
  '난리'],
 ['년', '생각', '못'],
 ['무슨', '중복', '경력', '단절', '건가'],
 ['세금',
  '일',
  '워킹맘',
  '죄인',
  '부모',
  '급여',
  '줄',
  '것',
  '언플',
  '원래',
  '지자체',
  '자체',
  '원금',
  '육아휴직',
  '수당',
  '원래',
  '월급',
  '한참',
  '못',
  '여기',
  '부모',
  '급여',
  '안',
  '신생아',
  '복직',
  '소리',
  '더',
  '육아휴직',
  '수당',
  '상한선',
  '신생아',
  '필수',
  '엄마',
  '휴직',
  '시기',
  '경제',
  '지원',
  '거',
  '면서',
  '무슨',
  '애'],
 ['기사', '놈', '사회'],
 ['국가', '자식', '왜', '국가', '세금', '퍼'],
 ['일',
  '고',
  '안주',
  '누가',
  '일',
  '이',
  '놈',
  '지원',
  '금은',
  '일',
  '꼬박꼬박',
  '세금',
  '사람',
  '쥐뿔'],
 ['육아휴직',
  '급여',
  '애',
  '정부',
  '지원',
  '아이',
  '더',
  '월급',
  '이외',
  '나라',
  '지원',
  '인식',
  '애',
  '부담'],
 ['육아휴직', '급여', '상한'],
 ['돈', '보지', '마구'],
 ['보건복지부', '말', '도하'],
 ['육아휴직', '급여', '상한', '최소', '은'],
 ['이건', '민주당', '발목', '선동'],
 ['내', '새끼', '분유', '값', '걱정', '어제', '일', '세상'],
 ['교육비',
  '문제',
  '우리나라',
  '교육',
  '

### 3. LDA 모델에 들어갈 객체(dictionary, corpus) 만들고 학습하기

In [8]:
#토픽모델링에 필요한 Dictionary 와 Corpus 만들기 (no_below = 20 빈도수 20이하 단어 제거)
id2word=corpora.Dictionary(data_word)
id2word.filter_extremes(no_below = 20) #20회 이하로 등장한 단어는 삭제
texts = data_word
corpus=[id2word.doc2bow(text) for text in texts]

In [9]:
import os
os.environ.update({'MALLET_HOME':r'C:/Users/User/mallet-2.0.8/'})

In [10]:
#gensim.models.wrappers.LdaMallet 모듈을 사용하면 MALLET의 (최적화 된 버전) 축소 된 깁스 샘플링을 사용하여 훈련 코퍼스에서 LDA 모델을 추정
# 다운로드 파일: http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
mallet_path = 'C:\\Users\\User\\mallet-2.0.8\\bin\\mallet' 
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=10, id2word=id2word)

In [11]:
ldamallet

In [12]:
text

,reply,policy
0,무슨 중복이요 그럼 휴직급여 그거 하나 받고 키우라는건디 다 일 그만두라는거에요 ...,영아수당 지급
1,와 년 주는거도 건드리네 더줄생각은 못하고,영아수당 지급
2,무슨 중복인지 그럼 다 경력단절되어야하는건가,영아수당 지급
3,세금 내고 일하는 워킹맘이 죄인인가 부모급여 줄 것처럼 언플해서 원래 지자체에서 자...,영아수당 지급
4,기사놈 사회성 없는거같음,영아수당 지급
...,...,...
2073,뭔 돈으로 준당가,영아수당 지급
2074,육아휴직 안해주는 회사는 어떡합니까그지같은 굥아,영아수당 지급
2075,애시기 낳으면 애국이라고 지껄이는것들 ㅇ나게 많네 명이상 낳는 부모들한테나 지급해라,영아수당 지급
2076,문재인정부 년간의 나랏빚 도덕적해이 기초연금폐지하고 이런자들만 독식해온 근로장려금폐...,영아수당 지급


### Coherence 점수를 계산하여 좋은 LDA 모델 찾기

In [13]:
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=texts, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()


def compute_coherence_values(dictionary, corpus, texts, limit, start=4, step=2):

    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=data_word, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values



# Can take a long time to run.
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=texts, start=4, limit=21, step=2)




limit=21; start=4; step=2;
x = range(start, limit, step)
topic_num = 0
count = 0
max_coherence = 0
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", cv)
    coherence = cv
    if coherence >= max_coherence:
        max_coherence = coherence
        topic_num = m
        model_list_num = count   
    count = count+1

        
# Select the model and print the topics
optimal_model = model_list[model_list_num]
model_topics = optimal_model.show_topics(formatted=False)
#print(optimal_model.print_topics(num_words=10))

def format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=texts):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    #ldamodel[corpus]: lda_model에 corpus를 넣어 각 토픽 당 확률을 알 수 있음
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num,topn=10)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']
    print(type(sent_topics_df))

    # Add original text to the end of the output
    #contents = pd.Series(texts)
    #sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    sent_topics_df = pd.concat([sent_topics_df, text['reply'],text['policy']], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=Data_list)

# Format
df_topic_tweet = df_topic_sents_keywords.reset_index()
df_topic_tweet.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'reply','policy']

# Show각 문서에 대한 토픽
#df_dominant_topic=df_dominant_topic.sort_values(by=['Dominant_Topic'])
#df_topic_tweet

# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)


topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()
topic_counts.sort_index(inplace=True)

topic_contribution = round(topic_counts/topic_counts.sum(), 4)
topic_contribution

lda_inform = pd.concat([sent_topics_sorteddf_mallet, topic_counts, topic_contribution], axis=1)
lda_inform.columns=["Topic_Num", "Topic_Perc_Contrib", "Keywords", "reply", "policy","Num_Documents", "Perc_Documents"]
lda_inform = lda_inform[["Topic_Num","Keywords","Num_Documents","Perc_Documents"]]
lda_inform
#lda_inform.Topic_Num = lda_inform.Topic_Num.astype(int)
lda_inform['Topic_Num'] =lda_inform['Topic_Num'] +1
lda_inform.Topic_Num = lda_inform.Topic_Num.astype(str)
lda_inform['Topic_Num'] =lda_inform['Topic_Num'].str.split('.').str[0]
df_topic_tweet['Dominant_Topic'] =df_topic_tweet['Dominant_Topic'] +1
df_topic_tweet.Dominant_Topic = df_topic_tweet.Dominant_Topic.astype(str)
df_topic_tweet['Dominant_Topic'] =df_topic_tweet['Dominant_Topic'].str.split('.').str[0]

Num Topics = 4  has Coherence Value of 0.3148250070795251
Num Topics = 6  has Coherence Value of 0.30942482074519095
Num Topics = 8  has Coherence Value of 0.29246541342009263
Num Topics = 10  has Coherence Value of 0.31522396724902524
Num Topics = 12  has Coherence Value of 0.3480794348588075
Num Topics = 14  has Coherence Value of 0.3474338676835088
Num Topics = 16  has Coherence Value of 0.34880121068959963
Num Topics = 18  has Coherence Value of 0.3597764023240837
Num Topics = 20  has Coherence Value of 0.34905328316181317


C:\Users\User\AppData\Local\Temp\ipykernel_18860\3300948403.py:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)


<class 'pandas.core.frame.DataFrame'>


### 5. 토픽 모델링 결과 확인하기 

In [14]:
lda_inform.to_csv ("result/lda_inform.csv", index = None)
lda_inform

,Topic_Num,Keywords,Num_Documents,Perc_Documents
0.0,1,"사람, 일, 안, 누가, 주지, 부부, 세금, 원금, 짓, 도대체",308,0.1483
1.0,2,"예산, 국가, 삭감, 대통령, 그, 줄, 곳, 정권, 미래, 위해",149,0.0717
2.0,3,"돈, 육아, 살, 개, 서울, 얼마나, 용돈, 손주, 자기, 도대체",94,0.0453
3.0,4,"거, 소득, 출산율, 지급, 이, 누구, 게, 임, 건가, 부부",123,0.0592
4.0,5,"나라, 진짜, 인간, 놈, 저, 정말, 데, 윤석열, 정신, 확인",163,0.0785
5.0,6,"만원, 집, 년, 결혼, 월, 값, 공무원, 보고, 때문, 아기",117,0.0563
6.0,7,"아이, 것, 수, 맞벌이, 생각, 조부모, 사회, 시간, 교육, 요즘",101,0.0486
7.0,8,"좀, 때, 어린이집, 제발, 비, 해, 학원, 엄마, 유치원, 학교",99,0.0477
8.0,9,"돈, 생각, 차라리, 막, 퍼주, 수급, 세상, 그냥, 퍼, 처",99,0.0477
9.0,10,"세금, 또, 내, 니, 연금, 빚, 눈, 그, 자기, 퍼",124,0.0597


#### 토픽 별 트윗 저장하기

In [18]:
#토픽별 트윗 저장
for i in range(1,topic_num+1):
    globals()['df_{}'.format(i)]=df_topic_tweet.loc[df_topic_tweet.Dominant_Topic==str(i)]
    globals()['df_{}'.format(i)].sort_values('Topic_Perc_Contrib',ascending=False,inplace = True)
    globals()['df_{}'.format(i)].to_csv ("./Result/topic("+str(i)+")_tweet.csv", index = None)

df_1

C:\Users\User\AppData\Local\Temp\ipykernel_18860\3468004463.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  globals()['df_{}'.format(i)].sort_values('Topic_Perc_Contrib',ascending=False,inplace = True)


,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,reply,policy
1390,1390,1,0.1895,"사람, 일, 안, 누가, 주지, 부부, 세금, 원금, 짓, 도대체",코로나로 외국인 노동자들 못들어와 지금 농촌은 부부 기준 월급이 까지 치솟았다 젊은...,영아수당 지급
16,16,1,0.1732,"사람, 일, 안, 누가, 주지, 부부, 세금, 원금, 짓, 도대체",일하는사람은 쎄빠지게 일하고 세금내고육휴급여 받으니 못준다 이러고집에 있는사람은 육...,영아수당 지급
123,123,1,0.1461,"사람, 일, 안, 누가, 주지, 부부, 세금, 원금, 짓, 도대체",그냥 젊은 부부는 일하지 말고 애들이나 낳고 키우라하고 월급을 주지세상 참 살기 힘...,영아수당 지급
1393,1393,1,0.1461,"사람, 일, 안, 누가, 주지, 부부, 세금, 원금, 짓, 도대체",돈만주면청년들도일할줄아나보지 그돈으로코인하지,영아수당 지급
1233,1233,1,0.1358,"사람, 일, 안, 누가, 주지, 부부, 세금, 원금, 짓, 도대체",이런 의미없는 지원 필요없다고등학교 동창 취업 한번 안하고 집 주변에는 자격증 공부...,영아수당 지급
...,...,...,...,...,...,...
988,988,1,0.0556,"사람, 일, 안, 누가, 주지, 부부, 세금, 원금, 짓, 도대체",물론 제대로 활용하는 사람도 일부있겠지만 대부분은 겆짓으로 서류꾸며서 돈받는다에 한...,영아수당 지급
989,989,1,0.0556,"사람, 일, 안, 누가, 주지, 부부, 세금, 원금, 짓, 도대체",참담하다,영아수당 지급
995,995,1,0.0556,"사람, 일, 안, 누가, 주지, 부부, 세금, 원금, 짓, 도대체",편법으로 그냥 돈받을수있는 정책이네요 차라리 육아수당을 더 줘요,영아수당 지급
999,999,1,0.0556,"사람, 일, 안, 누가, 주지, 부부, 세금, 원금, 짓, 도대체",우선 불공정이 심각하고 악용하는 부정수급자 태반이겠다 어차피 감시도 불가능하고 그...,영아수당 지급


#### 토픽 별 word cloud

In [28]:
from wordcloud import WordCloud
from konlpy.tag import Twitter
from collections import Counter

In [ ]:
#토픽별 word cloud
def flatten(l):
    flatList=[]
    for elem in l:
        if type(elem) == list:
            for e in elem:
                flatList.append(e)
        else:
            flatList.append(elem)
    return flatList


for i in range(1,topic_num+1):
    data_list = globals()['df_{}'.format(i)].reply.values.tolist()
    data_word=[]
    
    for j in range(len(data_list)):
        try:
            data_word.append(okt.nouns(data_list[j]))
        except Exception as e:
            continue       
    data_word=flatten(data_word)
    data_word=[x for x in data_word if not x.isdigit()]
    
    freq=pd.Series(data_word).value_counts().head(50)
    freq=dict(freq)
    
    wordcloud = WordCloud(font_path="./Font/BMHANNA_11yrs_ttf.ttf",
             relative_scaling = 0.2,
             background_color = 'white',
            ).generate_from_frequencies(freq)
    plt.figure(figsize=(16,8))
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.show()

    plt.savefig("./Result/topic("+str(i)+")wordcloud.png")

### 6. LDAvis로 토픽 모델 시각화

In [36]:
from gensim.models import LdaModel

def mallet_to_lda(optimal_model):
    model_gensim = LdaModel(
        id2word=optimal_model.id2word, num_topics=optimal_model.num_topics,
        alpha=optimal_model.alpha, eta=0, iterations=1000,
        gamma_threshold=0.001,
        dtype=np.float32
    )
    model_gensim.sync_state()
    model_gensim.state.sstats = optimal_model.wordtopics
    return model_gensim
model = mallet_to_lda(optimal_model)

import pyLDAvis
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(model, corpus, id2word)
vis

c:\Users\User\anaconda3\envs\test\lib\site-packages\pandas\core\internals\blocks.py:351: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
c:\Users\User\anaconda3\envs\test\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
c:\Users\User\anaconda3\envs\test\lib\site-packages\pandas\core\internals\blocks.py:351: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
15    -0.074935 -0.081813       1        1  5.589194
0     -0.058098 -0.081912       2        1  5.586070
8      0.407397  0.049045       3        1  5.578126
17    -0.071434  0.101791       4        1  5.573968
5     -0.054745 -0.133610       5        1  5.572096
4     -0.031889 -0.031673       6        1  5.562889
11     0.055333 -0.208777       7        1  5.561714
2      0.396508  0.125219       8        1  5.554643
1     -0.053505  0.004353       9        1  5.554257
14    -0.120852  0.215402      10        1  5.550550
10    -0.116558  0.171297      11        1  5.549043
7      0.013819 -0.065961      12        1  5.546612
16    -0.081187  0.067305      13        1  5.546093
3     -0.079491 -0.177290      14        1  5.540217
6      0.038820 -0.178219      15        1  5.540076
9      0.020788  0.014544      16        1  5.539195
12    -0.067735  0.014939      17        1  5.538600
13    -0.122234  0.195362      18        1  5.516658, topic_info=   Term   Freq       Total Category  logprob  loglift
37    돈  623.0  623.000000  Default     30.0     30.0
20    애  356.0  356.000000  Default     29.0     29.0
16   세금  290.0  290.000000  Default     28.0     28.0
33   사람  256.0  256.000000  Default     27.0     27.0
15   부모  251.0  251.000000  Default     26.0     26.0
..  ...    ...         ...      ...      ...      ...
17   소리    0.0   46.087685  Topic18     -inf     -inf
18   수당    0.0  135.733644  Topic18     -inf     -inf
19    안    0.0   71.598881  Topic18     -inf     -inf
20    애    0.0  356.934288  Topic18     -inf     -inf
21   엄마    0.0   29.335202  Topic18     -inf     -inf

[570 rows x 6 columns], token_table=      Topic      Freq Term
term                      
124       1  0.976876   가정
124       6  0.022718   가정
40        5  0.994812    값
142       4  0.029385    개
142       8  0.822768    개
...     ...       ...  ...
72       12  0.609773    해
83        1  1.007179   해결
54        5  1.002273   현실
65       17  1.000486   혜택
118       6  1.006616   확인

[289 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[16, 1, 9, 18, 6, 5, 12, 3, 2, 15, 11, 8, 17, 4, 7, 10, 13, 14])